In [5]:
import json
import shutil
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

In [6]:
gis = pd.read_csv('sites/gishwati_chimp_video_data.csv', encoding = "ISO-8859-1")
nnp = pd.read_csv('sites/nyungwe_chimp_video_data.csv', encoding = "ISO-8859-1")
df = pd.concat([gis,nnp])

In [7]:
df.columns

Index(['new_row_id', 'country', 'research_site', 'genus', 'species',
       'cam_coverage_area', 'location_metadata', 'habitat', 'utm_zone',
       'utm_long', 'utm_lat', 'cam_id', 'vid_res', 'vid_duration',
       'panaf_datasheet_comments', 'year', 'month', 'day', 'time_hr',
       'time_min', 'cell_id', 'subfolder', 'video_file_name', 'event_id',
       'min_number_chimps_per_video', 'max_number_chimps_per_video',
       'age_class', 'sex', 'tool_use', 'vocalization', 'bipedal',
       'camera_reaction', 'behavioral_context', 'other_species',
       'additional_comments', 'record_type'],
      dtype='object')

In [8]:
df.subfolder

0      gis_vid1_762418_9799548_20111103
1      gis_vid2_762955_9797814_20110713
2      gis_vid2_762955_9797814_20110713
3      gis_vid2_762955_9797814_20110804
4      gis_vid2_762955_9797814_20110804
                     ...               
473           nnp_vidukn_13_13_20131009
474           nnp_vidukn_13_13_20131009
475           nnp_vidukn_13_13_20131009
476           nnp_vidukn_13_13_20131009
477           nnp_vidukn_13_13_20131009
Name: subfolder, Length: 750, dtype: object

In [9]:
df.video_file_name

0      pict0002
1      pict0144
2      pict0144
3      pict0032
4      pict0032
         ...   
473    ek000008
474    ek000010
475    ek000010
476    ek000011
477    ek000012
Name: video_file_name, Length: 750, dtype: object

In [10]:
df.video_file_name[0]

0    pict0002
0    ek000049
Name: video_file_name, dtype: object

In [11]:
df['subdir_video'] = df['subfolder'] + '_' + df['video_file_name'].apply(lambda x: str(x))
df['subdir_video'] = df['subdir_video'].str.lower()

df['subdir_video_0'] = df.subdir_video.str.split('_').str[:5] + pd.Series('0' + df.subdir_video.str.split('_').str[5]).apply(lambda x: [x])
df['subdir_video_0'] = df['subdir_video_0'].str.join('_') 

# Only videos with 1 ape
df = df[df.max_number_chimps_per_video==1]
len(df)

157

In [14]:
# What we have locally
local_filenames = glob('/home/dl18206/Desktop/phd/data/panaf/chimp_videos/rwanda/**/*.mp4', recursive=True)
local_filenames = [x.split('/')[-2:] for x in local_filenames]
local_filenames = [x[-1].split('.')[0].lower() for x in local_filenames]

In [15]:
df[df.subdir_video.isin(local_filenames)].behavioral_context.value_counts().sum()
df[df.subdir_video_0.isin(local_filenames)].behavioral_context.value_counts().sum()
print(f"Total: {df[df.subdir_video.isin(local_filenames)].behavioral_context.value_counts().sum() + df[df.subdir_video_0.isin(local_filenames)].behavioral_context.value_counts().sum()}")

Total: 151


In [16]:
# Don't have...
df[~df.subdir_video_0.isin(local_filenames) & ~df.subdir_video.isin(local_filenames)]

,new_row_id,country,research_site,genus,species,cam_coverage_area,location_metadata,habitat,utm_zone,utm_long,...,tool_use,vocalization,bipedal,camera_reaction,behavioral_context,other_species,additional_comments,record_type,subdir_video,subdir_video_0
154,155,rwanda,gishwati,Pan,troglodytes schweinfurthii,19.20,treefall,forest - young secondary,35n,764192.0,...,no,no,no,yes,feeding,NaN,feeding on decaying wood ??,60s_video,gis_vid4_764192_9798159_20110901_pict0008,gis_vid4_764192_9798159_20110901_0pict0008
59,60,rwanda,nyungwe,Pan,troglodytes schweinfurthii,11.16,trail,"forest - mixed, closed understorey",35s,748181.0,...,no,no,no,yes,travel,NaN,NaN,60s_video,nnp_vid10_0748181_9728906_20150506_ek000007,nnp_vid10_0748181_9728906_20150506_0ek000007
333,334,rwanda,nyungwe,Pan,troglodytes schweinfurthii,10.54,feeding site,"forest - mixed, open understorey",35s,747239.0,...,no,no,no,yes,climbing,NaN,NaN,60s_video,nnp_vid34_0747239_9728367_20150319_ek000047,nnp_vid34_0747239_9728367_20150319_0ek000047
342,343,rwanda,nyungwe,Pan,troglodytes schweinfurthii,10.54,feeding site,"forest - mixed, open understorey",35s,747239.0,...,no,no,no,yes,climbing,NaN,NaN,60s_video,nnp_vid34_0747239_9728367_20150319_ek000066,nnp_vid34_0747239_9728367_20150319_0ek000066
390,391,rwanda,nyungwe,Pan,troglodytes schweinfurthii,8.71,feeding site,"forest - mixed, open understorey",35s,748944.0,...,no,no,no,yes,travel,NaN,NaN,non_60s_video,nnp_vid37_0748944_9728646_20150622_ek000017,nnp_vid37_0748944_9728646_20150622_0ek000017
393,394,rwanda,nyungwe,Pan,troglodytes schweinfurthii,8.71,feeding site,"forest - mixed, open understorey",35s,748944.0,...,no,yes,no,no,climbing,NaN,NaN,non_60s_video,nnp_vid37_0748944_9728646_20150622_ek000019,nnp_vid37_0748944_9728646_20150622_0ek000019
